## *Задание 1*
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [44]:
import pandas as pd
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.tail(100)

,userId,movieId,rating,timestamp
99904,671,590,4.0,1065149296
99905,671,608,4.0,1064890575
99906,671,745,4.0,1065149085
99907,671,919,4.0,1065149458
99908,671,1035,5.0,1065149492
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [45]:
users_rating_count = ratings.groupby('userId')[['userId']].count()
users_rating_count_100 = (users_rating_count > 100).values
users_rating_min = ratings.groupby('userId')[['timestamp']].min()
users_rating_max = ratings.groupby('userId')[['timestamp']].max()

In [46]:
users_rating_count = users_rating_count[users_rating_count_100]
users_rating_min = users_rating_min[users_rating_count_100]
users_rating_max = users_rating_max[users_rating_count_100]

In [47]:
mean_lifetime = (users_rating_max - users_rating_min).mean()
print(mean_lifetime)
lifetime = (users_rating_max - users_rating_min)
users_rating_count.join(lifetime).mean()

timestamp    4.008051e+07
dtype: float64


userId       3.119651e+02
timestamp    4.008051e+07
dtype: float64

## *Задание 2*
   Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [50]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [55]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [52]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [53]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [67]:
rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer')

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [70]:
rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer').merge(client_base, on='client_id', how='outer')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


## *Задание 3*

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


In [108]:
user_base = pd.DataFrame(
    {
        'user_id': [111, 111, 112, 113, 113, 113, 114, 115, 116, 117, 118],
        'visit_id': [1, 2, 1, 1, 2, 3, 1, 1, 1, 1, 1],
        'buy': [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0],
        'latitude': [37.1, 37.5, 37.5, 37.5, 37.5, 37.4, 37.7, 37.3, 37.5, 37.3, 37.5],
        'longitude': [55.4, 55.5, 55.4, 55.7, 55.2, 55.4, 55.8, 55.3, 55.5, 55.5, 55.4],
    }
)
user_base

,user_id,visit_id,buy,latitude,longitude
0,111,1,1,37.1,55.4
1,111,2,1,37.5,55.5
2,112,1,0,37.5,55.4
3,113,1,0,37.5,55.7
4,113,2,1,37.5,55.2
5,113,3,1,37.4,55.4
6,114,1,0,37.7,55.8
7,115,1,0,37.3,55.3
8,116,1,1,37.5,55.5
9,117,1,1,37.3,55.5


#### Признаки

In [120]:
# Можно рассчитывать пройденное расстояние каждым пользователем
import numpy as np
def dist(x):
    return np.sqrt(np.dot(x.latitude, x.latitude) - 
                   (np.dot(x.latitude, x.longitude) + np.dot(x.latitude, x.longitude)) + 
                   np.dot(x.longitude, x.longitude))

group_user = user_base.groupby('user_id')
group_user.apply(dist)

user_id
111    25.668853
112    17.900000
113    31.121215
114    18.100000
115    18.000000
116    18.000000
117    18.200000
118    17.900000
dtype: float64

In [ ]:
# А также простые статистики для каждого пользователя и всей выборки в целом,
# Средние пройденные расстояния, дисперсии,и тд
# Координаны где сделано больше всего покупок, 
# Наиболее часто посещаемые координанат
# Наиболее частые маршруты

